# Numerische Methoden für Systeme

### 1.3 Verständnisfragen

1. Was verstehen Sie unter einem konservativen numerischen Verfahren?


 2. Das klassische Runge-Kutta-Verfahren ist ”nahezu” konservativ. Was bedeutet das? Worauf müssen Sie achten,
 wenn Sie das Runge-Kutta-Verfahren für die Lösung von Stabilitätsproblemen einsetzen wollen.

### 1.4 Aufgaben


### 1. Differentialgleichung und Anfangswertproblem

### 2. gDgln lösen mit RK4 Verfahren

### 3. Optional

### 4. gDgln lösen mit Impliziten Mittelpunktregel

### 5. Vergleichen der Numerischen Lösungen